# Bossa Muffin Scrapping DuckDuckGo Program v3 - 24/04/2022 
# With Requests modul

## Install all package i need for this program

## Don't execute if not need
'''
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install fake_useragent
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install time
!{sys.executable} -m pip install json
!{sys.executable} -m pip install pandas
'''

Import all moduls i need for this program

In [ ]:
import requests
from fake_useragent import UserAgent 
from bs4 import BeautifulSoup 
import time
import random
import re
import csv
import json
import pandas as pd

## A class to prepare the research

In [ ]:
# URL_TO_TEST_MY_IP = 'https://httpbin.org/ip'

In [ ]:
# Research parameters and arguments 
class TheResearch:
    
    def __init__(self, words_list):
        self.POST_PARAMS = ['q', 's', 'nextParams', 'v', 'l', 'o', 'json', 'dc', 'api', 'vdq', 'kl' ]
        # interesting nodes in the research soup response
        self.CLASS_TARGETS = ['result__a', 'result__link', 'result__snippet']
        # Evite de faire toujours la même recherche dans DDG pedant les tests -> diminue le risque de blocage serveur 
        # The payload search request
        self.PAYLOAD = {
            'q': random.choice(words_list),
            'b': '',
            'kl': 'fr-fr'
        }


In [ ]:
words_list = ['France', 'Président 2022', 'Macron', 'Election', 'Marine Lepen score', 'taux d\'Abstention', 'élection présidentielle', 'programme Macron', 'rapport GIEC 2022', 'résultat élection']
search = TheResearch(words_list)

In [ ]:
class Scrap_session:
    
    def __init__(self, payload, url = "https://html.duckduckgo.com/html"):
        self.URL = url
        ## Response dict
        self.responses = {}
        ## Soup string 
        self.soup = ''
        ## Pages counter
        self.page= 0
        ## Number of scraped nodes
        self.nb_nodes = 1
    
    # Generate a strong user agent for the session
    # Example of a full payload : payload = { 'q': 'test', 's': '29', 'nextParams':'', 'v': 'l', 'o': 'json', 'dc': '32', 'api': 'd.js', 'vqd': 'default', 'kl': 'wt-wt' }
    def _defineHeader(self):
        ua = UserAgent().random
        return {'user-agent': ua}
        
    # Import the page results from the given URL and datas in post method by payload
    def getPage(self, proxy, post_mod=False):
        r = False
        ipport = proxy['ip']+':' + proxy['port']
        header = self._defineHeader()
        
        if post_mod == False:
            try:
                self.responses[self.page] = s.get(self.URL, 
                                                  proxies={'http': ipport , 'https': ipport}, 
                                                  timeout = proxy['timeout'], 
                                                  headers = header
                                                 )
                r=True
            except:
                print('GET : Failed to join the server via : ', ipport)
            '''
            print('Status code', self.responses[self.page].status_code)
            print('Headers\' page', slef.responses[self.page].headers['content-type'])
            print('Cookies of the session n°', page, 'is :', s.cookies.get_dict())
            print('Current payload :', search.payload)
            '''
        else:
            try:
                self.responses[self.page] = s.post(self.URL, 
                                                   proxies={'http': ipport , 'https': ipport}, 
                                                   timeout=proxy['timeout'], 
                                                   data=search.PAYLOAD, 
                                                   cookies=self.responses[self.page-1].cookies, 
                                                   headers=self.header
                                                  ) 
                r=True
            except:
                 print('GET : Failed to join the server via proxy : ', ipport)

        return r

In [ ]:
scrap = Scrap_session(search.PAYLOAD, "https://html.duckduckgo.com/html")

## A class to manage different proxies to use in the scrapping process

In [ ]:
class Proxies:
    
    def __init__(self, proxies_csv = 'proxies_list.csv'):
        self._PROXIES_FILE = proxies_csv
        self.LIST_TO_USE = self._extractProxiesList(self._PROXIES_FILE)
    

    # Transform string milisecond '1389 ms' in an int second '1.5'
    def _strMsToIntS(self, str_ms):
        int_ms=int(str_ms.replace(' ms', ''))
        #print(int_ms)
        s=(int_ms//1000)*1.5
        #print(s)
        return int_ms
    
    def _extractProxiesList(self, file):
        proxies_list = []
        with open(file, 'r') as f:
            reader_f = csv.reader(f)
            proxies_list = [{'ip':row[0], 'port':row[1], 'timeout':self._strMsToIntS(row[3]), 'prot':row[2], 'last_status':0} for row in reader_f]
        return proxies_list

In [ ]:
proxies = Proxies('proxies_list.csv')

In [ ]:
# Show an example result of the miliseconds string to seconds int
proxies._strMsToIntS('100 ms')

In [ ]:
def formatResults(soup):
    # Select the payload tab prepare to post method
    nodes_payload={}

    for param in search.POST_PARAMS:
        nodes_payload[param] = scrap.soup.find_all(attrs={"name": param})
        if len(nodes_payload[param]) != 0:
            print('Taille!=0 : ', len(nodes_payload[param]))
            print('POP : ', nodes_payload[param].pop())
            search.PAYLOAD[param] = nodes_payload[param].pop()['value']
        else:
            search.PAYLOAD[param] = ''

    # Select the interesting object
    nodes = {}
    for target in search.CLASS_TARGETS:
        nodes[target] = scrap.soup.find_all(class_=target)

    nb_nodes = len(nodes[class_targets[0]])
    print('nb results', nb_nodes)
    
    ## ------------------------------------------------
    ## Format intersting elements in a shaped tab / bdd
    ## ------------------------------------------------
    tab_results = []
    
    for i in range(nb_nodes):
        result = {}
        result['title'] = nodes_titles[i].text
        result['link'] = nodes_links[i].text
        result['text'] = nodes_texts[i].text
        tab_results.append(result)
        
    return tab_results

In [ ]:
def saveResultsInCsvFile(tab_results : dict) -> str:
    bdd_results = pd.DataFrame(tab_results, columns=['title', 'link', 'text'])

    print('Number of elements : ', scrap.nb_nodes)
    print('\n')
    print(bdd_results)
    
    ## ------------------------------------------------
    ## Write the tab in a CSV file
    ## ------------------------------------------------
    clean_word_request = removeSpecialChars(payload['q'])
    csv_name_file = 'duckduckgo_requests_results_' + clean_word_request + '.csv'
    bdd_results.to_csv(csv_name_file)

    return csv_name_file

In [ ]:
def removeSpecialChars(payload_word_request: str) -> str:
    payload_word_request = re.sub('[^a-zA-Z0-9 \n\.]', '_', payload_word_request)
    return payload_word_request.replace(" ", "")

In [ ]:
#print(removeSpecialChars('hé_ )°ho'))

## Open a virgin session with the website on the landing html page

In [ ]:
def scrapResultsPage():
    n=0
    while scrap.nb_nodes != 0 and scrap.page < 10 and n < len(proxies.LIST_TO_USE) :
        ## Don't flood the server !!!!
        time.sleep(3)
        print('\n')
        print('Ready to scrap the page n°', scrap.page)

        # We test the proxies_list until find a working proxy to do the request on th target page
        while n < len(proxies.LIST_TO_USE) and proxies.LIST_TO_USE[n]['last_status'] != 200 :
            time.sleep(0.1)
            print('\n')
            print('Proxy n°', n, ' -> ', proxies.LIST_TO_USE[n]['ip'],':', proxies.LIST_TO_USE[n]['port'])
            ## Import the page results from the given URL 
            ## datas in payload (as search request) are in the post method
            request_success = scrap.getPage(proxies.LIST_TO_USE[n], post_mod=True)
            
            try:
                proxies.LIST_TO_USE[n]['last_status'] = scrap.responses[scrap.page].status_code
                
            except KeyError:
                request_success = False
                proxies.LIST_TO_USE[n]['last_status'] = 0
                print('>	PROXY ERROR')
                print('>	Results page is unscrapable with this script')
                print('>	Define status code to 0 on page n°', scrap.page)
                # Change proxy if current proxy fail
                n+=1

            if request_success and proxies.LIST_TO_USE[n]['last_status'] == 200:
                # The proxy n works !
                # The imported code is put in a soup object, easy to work on
                scrap.soup = BeautifulSoup(scrap.responses[scrap.page].text, "html.parser")
                dict_results = formatResults(scrap.soup)
                saveResultsInCsvFile(dict_results)
                ## Now, we scrap a new page
                scrap.page += 1

            elif request_success and proxies.LIST_TO_USE[n]['last_status'] != 200:
                print('>	REQUEST ERROR')
                print('>	Results page is unscrapable with this script')
                print('>	Status code :', proxies.LIST_TO_USE[n]['last_status'], ' on page n°', scrap.page)
                # Change proxy if current proxy fail
                n+=1

### Delay to pass th 403 error

In [ ]:
#time.sleep(360)

## Execute the script

In [ ]:
with requests.Session() as s:
    scrapResultsPage()
print('\n')
print(f"End of Requests scrapping on : {scrap.URL} with search : '{search.PAYLOAD['q']}'")
print('\n')

In [ ]:
exit('End') 